In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pyproj
import geopandas as gpd
from shapely.geometry import Point
from sklearn.linear_model import Lasso,LassoCV,Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [3]:
concat_data = pd.read_csv("new_concat_data.csv")

臺北市、新北市、桃園市、臺中市、臺南市、高雄市、(新竹縣、新竹市)

In [107]:
for i in unique_city:
    print(i, concat_data[concat_data['縣市'] == i].shape)

台北市 (1265, 60)
高雄市 (1310, 60)
新北市 (4156, 60)
桃園市 (1165, 60)
台中市 (982, 60)
台南市 (474, 60)
苗栗縣 (43, 60)
新竹縣 (347, 60)
基隆市 (222, 60)
屏東縣 (46, 60)
新竹市 (243, 60)
宜蘭縣 (75, 60)
花蓮縣 (12, 60)
嘉義市 (25, 60)
金門縣 (30, 60)
嘉義縣 (2, 60)
彰化縣 (20, 60)
雲林縣 (2, 60)


In [4]:
# 六都分一類
Taipei = concat_data[concat_data['縣市'] == "台北市"]
Kaohsiung = concat_data[concat_data['縣市'] == "高雄市"]
New_Taipei = concat_data[concat_data['縣市'] == "新北市"]
Taoyuan = concat_data[concat_data['縣市'] == "桃園市"]
Taichung = concat_data[concat_data['縣市'] == "台中市"]
Tainan = concat_data[concat_data['縣市'] == "台南市"]
Hsinchu = concat_data[(concat_data['縣市'] == "新竹縣") | (concat_data['縣市'] == "新竹市")]
Else = concat_data[(concat_data['縣市'] != "台北市") & (concat_data['縣市'] != "高雄市") & (concat_data['縣市'] != "新北市") & (concat_data['縣市'] != "桃園市") & (concat_data['縣市'] != "台中市") &( concat_data['縣市'] != "台南市") & (concat_data['縣市'] != "新竹縣") & (concat_data['縣市'] != "新竹市")]
multi = [Taipei, Kaohsiung, New_Taipei, Taoyuan, Taichung, Tainan, Hsinchu, Else]

In [5]:
Taipei.shape[0] + Kaohsiung.shape[0] + New_Taipei.shape[0] + Taoyuan.shape[0] + Taichung.shape[0] + Tainan.shape[0] + Else.shape[0] + Hsinchu.shape[0] == concat_data.shape[0]

True

In [8]:
z_scaler = preprocessing.StandardScaler()

def GetSplit(data):
    train_data, test_data = train_test_split(specific, random_state=777, train_size=0.8)

    Y_train = train_data[['單價']]
    X_train = train_data.drop(['單價'], axis=1)
    Y_test = test_data[['單價']]
    X_test = test_data.drop(['單價'], axis=1)
        
    return X_train, Y_train, X_test, Y_test

def ClusterTrainer(city):
    
    # specific = concat_data[concat_data['縣市'] == city]
    specific = city
    
    specific = specific[[
        '縣市', '鄉鎮市區', '路名', '主要用途', '主要建材', '建物型態', '使用分區', '備註', '單價',
        'lng', '最近航空站', '最近幼稚園、托兒所',
        '屋齡', '最近一般公園', '最近碼頭', '最近大型購物中心', 
        '最近超級市場', '最近大學及研究所', '總樓層數',
        '最近國中', '最近生活百貨量販',
        '最近捷運站', '最近電力資源', '最近高鐵站',
        '最近產業用水及設施', '最近飯店', '最近觀光遊憩設施', 
        '最近主要道路建設', 'lat', '最近里鄰公園', '最近娛樂設施', 
        '國中(高)', '國中(中)', '最近火車站', '車位面積', 
        '最近百貨公司', '陽台面積', '移轉層次', '最近停車場地', 
        '最近醫療保健', '最近金融機構', '最近加油站', 
        '土地面積', '主建物面積', '建物面積', '最近工廠', 
        '最近傳統市場', '附屬建物面積', '國中(下)', 
        '最近高中', '最近商店', '最近專科', '最近國小', 
        '最近廟宇距離', '最近金融機構距離'
    ]]
    specific.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(specific.iloc[:, 9:]), index = specific.index)
    
    specific = pd.get_dummies(specific, columns=['鄉鎮市區','路名','主要用途','主要建材','建物型態','使用分區','備註'])
    specific = specific.drop(['縣市'], axis=1)
    
    train_data, test_data = train_test_split(specific, random_state=777, train_size=0.8)

    # train_test_data = GetSplit(specific)
    Y_train = train_data[['單價']]
    X_train = train_data.drop(['單價'], axis=1)
    Y_test = test_data[['單價']]
    X_test = test_data.drop(['單價'], axis=1)

    ridge = Ridge()
    parameters = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
    ridge_reg = GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
    result = ridge_reg.fit(X_train.values, Y_train['單價'].values)
    
    return result, Y_test, X_test

In [9]:
test_res_list = []

for i in multi:
    res = ClusterTrainer(i)
    predictions = res[0].predict(res[2].values)
    test_res_list.append(np.mean(np.abs((res[1]['單價'].values - predictions) / res[1]['單價'].values)) * 100)
    np.mean(np.abs((res[1]['單價'].values - predictions) / res[1]['單價'].values)) * 100

/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_20270/1308320488.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(specific.iloc[:, 9:]), index = specific.index)
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_20270/1308320488.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(specific.iloc[:, 9:]), index = specific.index)
/var/folders/w2/_g9w5yys0f171q4qqm469z1h00

In [10]:
print(test_res_list)
print(np.mean(test_res_list))

[8.198594057570123, 11.011285443102144, 7.844658313722652, 9.317758715058845, 10.771101816952354, 13.536672363781637, 14.579456129931417, 11.47993508947549]
10.842432741199332


## RF

In [21]:
def ClusterTrainer(city):
    
    # specific = concat_data[concat_data['縣市'] == city]
    specific = city
    
    specific = specific[[
        '縣市', '鄉鎮市區', '路名', '主要用途', '主要建材', '建物型態', '使用分區', '備註', '單價',
        'lng', '最近航空站', '最近幼稚園、托兒所',
        '屋齡', '最近一般公園', '最近碼頭', '最近大型購物中心', 
        '最近超級市場', '最近大學及研究所', '總樓層數',
        '最近國中', '最近生活百貨量販',
        '最近捷運站', '最近電力資源', '最近高鐵站',
        '最近產業用水及設施', '最近飯店', '最近觀光遊憩設施', 
        '最近主要道路建設', 'lat', '最近里鄰公園', '最近娛樂設施', 
        '國中(高)', '國中(中)', '最近火車站', '車位面積', 
        '最近百貨公司', '陽台面積', '移轉層次', '最近停車場地', 
        '最近醫療保健', '最近金融機構', '最近加油站', 
        '土地面積', '主建物面積', '建物面積', '最近工廠', 
        '最近傳統市場', '附屬建物面積', '國中(下)', 
        '最近高中', '最近商店', '最近專科', '最近國小', 
        '最近廟宇距離', '最近金融機構距離'
    ]]
    specific.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(specific.iloc[:, 9:]), index = specific.index)
    
    specific = pd.get_dummies(specific, columns=['鄉鎮市區','路名','主要用途','主要建材','建物型態','使用分區','備註'])
    specific = specific.drop(['縣市'], axis=1)
    
    train_data, test_data = train_test_split(specific, random_state=777, train_size=0.8)

    # train_test_data = GetSplit(specific)
    Y_train = train_data[['單價']]
    X_train = train_data.drop(['單價'], axis=1)
    Y_test = test_data[['單價']]
    X_test = test_data.drop(['單價'], axis=1)

    rf_regressor = RandomForestRegressor()
    result = rf_regressor.fit(X_train, Y_train)
    
    return result, Y_test, X_test

In [22]:
test_res_list = []

for i in multi:
    res = ClusterTrainer(i)
    predictions = res[0].predict(res[2].values)
    test_res_list.append(np.mean(np.abs((res[1]['單價'].values - predictions) / res[1]['單價'].values)) * 100)
    np.mean(np.abs((res[1]['單價'].values - predictions) / res[1]['單價'].values)) * 100

/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_20270/869066144.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(specific.iloc[:, 9:]), index = specific.index)
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_20270/869066144.py:37: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = rf_regressor.fit(X_train, Y_train)
/Users/wangqiqian/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/var/folders/w2/_g9w5yys0f171q4qqm4

In [23]:
print(test_res_list)
print(np.mean(test_res_list))

[7.306811463469909, 10.643745947128162, 7.56982350272723, 8.257744953390503, 9.832314334637289, 12.397633435720234, 15.44514262785683, 11.162372156672976]
10.32694855270039
